In [142]:
import cv2
from keras.models import load_model
import numpy as np

In [160]:
# Load the pre-trained face detection cascade classifier
face_cascade = cv2.CascadeClassifier('../haarcascade_frontalface_default.xml')

# Load the pre-trained emotion detection model
emotion_model = load_model('../fer_model.h5')

# Read the input image
img = cv2.imread('../inputs/Thesis/surprise.png')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [161]:
# Loop through each face in the image
for (x, y, w, h) in faces:
    # Extract the face region of interest
    roi_gray = gray[y:y+h, x:x+w]
    
    # Resize the ROI to match the input shape of the emotion detection model
    roi_gray = cv2.resize(roi_gray, (48, 48))
    
    # Normalize the pixel values of the ROI to be between 0 and 1
    roi_gray = roi_gray / 255.0
    
    # Make a prediction on the ROI using the emotion detection model
    prediction = emotion_model.predict(roi_gray.reshape(1, 48, 48, 1))
        
    np.save('image_emotion.npy', prediction)
    
    # Get the emotion label with the highest probability
    emotion_label = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'][np.argmax(prediction)]
    
    print(prediction)
    
    # Draw a rectangle around the face and label the detected emotion
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(img, emotion_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

1/1 [==============================] - 0s 94ms/step
[[0.3929982  0.0364162  0.02769915 0.04602179 0.3984292  0.09677131
  0.00166411]]
1/1 [==============================] - 0s 12ms/step
[[0.09342158 0.01175559 0.1619229  0.24311104 0.05837929 0.3508038
  0.0806058 ]]


In [136]:
#resize the image to the screen size
screen_res = 1920, 1080
img = cv2.resize(img, screen_res)

In [162]:
# Display the output image
cv2.imshow('Emotion Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()